## Etivity 4 - Tensors

### Student Name:   Cathal Cronin

### Student ID:     10131531
___
**Problem Description:**

With the final version of the code from the peer you selected in E-tivity 3 as a starting point, create a class that contains one tensor and provides the following methods:

* Calculate the size of the tensor as a 2D tuple
* Sum/subtract the tensor with another tensor of suitable size
* Multiply the tensor with a tensor of suitable size
* Calculate the determinant of the tensor, but only for tensors of size 2x2 (matrix)
* Calculate the inverse of the tensor, but only for tensors of size 2x2
* Calculate the cross product of the tensor with another tensor of suitable size

___
**Guidelines**

* Write your code without the use of imported modules.
* Provide suitable exception handling to prevent the above operations from being used on unsuitable tensors.
* Test your code with one example for each of the following scenarios:
* sum of a 4x4 with a 4x4 matrix
* multiplication of a 4x4 with a 4x4 matrix
* multiplication of a 4x4 matrix with a suitable vector
* determinant of a 2x2 matrix
* inverse of a 2x2 matrix
* cross-product of 2 suitable tensors
___

For E-tivity 3 I reviewed Mark Murnane's code. Available [here](https://gitlab.com/CE4021/Group2/week3/blob/MarkMurnane/Etivity3_ID18195326.py).

#### Approach
I started with Marks code as a base to start from. His code was easy to understand and easy to manipulate.

Using his class I added 3 new methods:

1. determinant()
2. inverse()
3. cross_product()

With these methods implemented, I added more tests to his list of tests. All of which cover the requirements above. 

One small change I made to marks code was to remove an imported module called Numbers. He was using to check that the contents of his tensors were valid integers and floats. Requirements listed no imported modules were allowed. The code change by removing this library was simply some extra 'if' checking on the datatype in the tensor by making use of the isinstance() method. 

### Code

In [1]:
"""
Matrix class pulled from week3 repo using Mark Murnane's branch as a starting point.

Renamed to Tensor to represent a more general view of vectors and matrices.
"""


class Tensor:
    """This class represents a mathematical Tensor for use in Linear Algebra

    A tensor is a structure of m x n elements, where m is the number of columns in the tensor and n is the number of rows.
    """

    _ROWS = 0
    _COLS = 1

    def __init__(self, element_data):
        """Creates a new instance of a Tensor.

        Args:
            element_data[[row_data]+]    A list/tuple containing other 1 or more lists/tuples representing each row of the tensor.  Each row must be the same length.

        Return:
            Tensor  Instance of the Tensor class, or None if the element_data is not valid.

        Raises:
            TypeError        If no parameters are provided or the list is empty
            ValueError       If the elements are not all numberic, or the rows are not the same length
        """

        # If element data is (neither a list or tuple) or is empty return an error
        if not (isinstance(element_data, list) or isinstance(element_data, tuple)) or not element_data:
            raise TypeError("Invalid/no parameters .")

        # Then make sure the list does actually contain some other non-empty lists
        elif not all(element_data):
            raise ValueError("Element data is incomplete.  Some rows are empty.")

        # Check that the rows are the same legnthlist elements are actually numbers
        else:
            for row in element_data:
                if not (isinstance(row, list) or isinstance(row, tuple)):
                    raise TypeError(f"Row is not a valid input type {row}")

                row_length = len(element_data[0])
                if len(row) != row_length:
                    raise ValueError("Rows are not a uniform length")

                if not all(isinstance(value, int) or isinstance(value, float) for value in row):
                    raise ValueError(f"Element data contains non-numeric data in row {row}")

        # By definition a tensor has a fixed number of rows and columns.
        # The order/dimensions is the number of rows and all rows will contain the same number of columns as the first row
        self.order = (len(element_data), len(element_data[0]))
        self.elements = tuple(tuple(row) for row in element_data)

    def get_order(self):
        """Returns a tuple with the order of the tensor, i.e. (rows x columns)."""
        return self.order

    def __str__(self):
        mat_str = ''
        for row in self.elements:
            row_str = ''
            for col_val in row:
                row_str = row_str + ' ' + str(col_val) + ' '
            mat_str = mat_str + row_str + '\n'

        return mat_str

    def _is_valid_add_sub_eq_tensor(self, other):
        """Internal helper function that checks if this tensor and other are valid for equality, addition or subtraction.

        To be valid for these operations both must be matrices with the same order.
        """

        if not isinstance(other, self.__class__):
            return False

        if self.get_order() != other.get_order():
            return False

        return True

    def _is_valid_tensor_multiplier(self, other):
        """Internal helper function that checks if this tensor and other can be validly multiplied.

        To be valid the number of columns in this tensor must equal the number of rows in other.
        """

        if not isinstance(other, self.__class__):
            return False

        if self.order[Tensor._COLS] != other.order[Tensor._ROWS]:
            return False

        return True

    def __eq__(self, other):
        """Compares this tensor object to another tensor object.  Returns true if the order of the tensor is the same and the elements at each position are the same."""

        if self._is_valid_add_sub_eq_tensor(other):
            for self_row, other_row in zip(self.elements, other.elements):
                if self_row != other_row:
                    return False
            return True
        else:
            return False

    def _apply_function_to_elements(self, other, lfunc):
        """Returns a list containing the results of the function lfunc applied to rows of both self and other.  Used for simple operations like addition and subtraction.

        The function represented by lfunc is required to take two arguments, representing element values from the tensor.

        """
        return [list(map(lfunc, self_row, other_row)) for self_row, other_row in zip(self.elements, other.elements)]

    def __add__(self, other):
        """Operator overload method for the + operator.  For tensor addition the other tensor must have the same order/dimension as this tensor.

        Calls _apply_function_to_elements, passing a lambda function (lambda x, y: x + y) that takes two parameters and returns their sum.
        """

        if self._is_valid_add_sub_eq_tensor(other):
            return Tensor(self._apply_function_to_elements(other, lambda x, y: x + y))
        else:
            return NotImplemented

    def __sub__(self, other):
        """Operator overload method for the - operator.  For tensor subtraction the other tensor must have the same order/dimension as this tensor.

        Calls _apply_function_to_elements, passing a lambda function (lambda x, y: x + y) that takes two parameters and returns their difference.
        """
        if self._is_valid_add_sub_eq_tensor(other):
            return Tensor(self._apply_function_to_elements(other, lambda x, y: x - y))
        else:
            return NotImplemented

    def _scalar_multiplication(self, other):
        """Internal method to multiply this tensor by a scalar value."""
        if isinstance(other, int):
            # Scalar multiplication uses list comprehension with a map to multiply each element by the scalar
            new_elements = [list(map(lambda element: element * other, row)) for row in self.elements]
            new_tensor = Tensor(new_elements)
            return new_tensor
        else:
            return NotImplemented

    def __mul__(self, other):

        """Operator overload for the * operator.  Multiplies this tensor by a scalar value.

        Scalar multiplication of a tensor is commutative.  This method handles the case of <tensor> * <scalar>.
        """
        return self._scalar_multiplication(other)

    def __rmul__(self, other):

        """Operator overload for the * operator.  Multiplies this tensor by a scalar value.

        Scalar multiplication of a tensor is commutative.  This method handles the case of <scalar> * <tensor>.
        """
        return self._scalar_multiplication(other)

    def __matmul__(self, other):
        """Operator overload for the @ operator.  Multiplies two matrices to return the tensor product.

        For tensor multiplication the number of rows in other must match the number of columns in this tensor.
        """

        # Make sure this is a valid multiplication
        if not self._is_valid_tensor_multiplier(other):
            return NotImplemented

        new_tensor = None

        # Tensor multiplication C=AB, i.e. c = (self)(other)

        # The order/dimension of the Product tensor is the rows from
        c_order = (self.order[Tensor._ROWS], other.order[Tensor._COLS])

        # Initialise the elements array to an m*n array of 0s
        c = [[0] * c_order[Tensor._COLS] for i in range(c_order[Tensor._ROWS])]

        # For each position in the new tensor c, calculate the values from this tensor and other
        # For any C[i][k] = Sum over j of (A[i][j]*B[j][k])
        for i in range(c_order[Tensor._ROWS]):
            for k in range(c_order[Tensor._COLS]):
                #                 c[i][k] = sum(self.elements[i][j] * other.elements[j][k]) for j in range(self.order[Tensor._COLS])
                for j in range(self.order[Tensor._COLS]):
                    c[i][k] += (self.elements[i][j] * other.elements[j][k])

        new_tensor = Tensor(c)

        return new_tensor

    def determinant(self):
        """Calculate the determinant of the tensor. Note tensor must be square and of size 2 by 2

        Returns: (int) Integer representing the determinant. Can be postive or negative
        """
        rows, cols = self.order
        if rows != cols:
            raise ValueError("Matrix is non-square")
        elif rows != 2:
            raise NotImplementedError("Only 2x2 matrices are supported at this time.")

        return (self.elements[0][0] * self.elements[1][1]) - (self.elements[1][0]) * self.elements[0][1]

    def inverse(self):
        """Calculate the inverse of the tensor. Note only works for matrices of size 2 by 2

        Returns: The inverse tensor, can be multiplied by the orignal tensor to get an identity tensor.
                 None: If the determinant is 0 meaning an inverse of this tensor cannot be calcualted.
        """
        determinant = self.determinant()
        if determinant == 0:
            return None

        a = self.elements[0][0] / determinant
        b = self.elements[0][1] / determinant
        c = self.elements[1][0] / determinant
        d = self.elements[1][1] / determinant

        return Tensor([(d, -b), (-c, a)])

    def cross_product(self, vector_b):
        """Calculate the cross product for 2 vectors. Vectors must be in 3 dimensions.

        Returns: (Tensor) A new vector representing the cross product of the A X B
        """

        if not isinstance(vector_b, self.__class__):
            raise TypeError("Cross Product is only defined in terms of two tensors")

        vector_a_rows, vector_a_cols = self.order
        vector_b_rows, vector_b_cols = vector_b.get_order()

        if vector_a_rows != 3 or vector_b_rows != 3 or vector_a_cols != 1 or vector_b_cols != 1:
            raise ValueError("Error: Vectors must be in 3 dimensions")
        else:
            u1 = self.elements[0][0]
            u2 = self.elements[1][0]
            u3 = self.elements[2][0]

            v1 = vector_b.elements[0][0]
            v2 = vector_b.elements[1][0]
            v3 = vector_b.elements[2][0]

            result = ([u2 * v3 - u3 * v2], [u3 * v1 - u1 * v3], [u1 * v2 - u2 * v1])
            cross_product_tensor = Tensor(result)

        return cross_product_tensor


### Testing File 
___
##### Contians a list of tests to cover the requirements 
- sum of a 4x4 with a 4x4 matrix
- multiplication of a 4x4 with a 4x4 matrix
- multiplication of a 4x4 matrix with a suitable vector
- determinant of a 2x2 matrix
- inverse of a 2x2 matrix
- cross-product of 2 suitable tensors

In the repo are hand writted tests I have done to which will match the results of the listed tests below.

#### Test Output
![](https://gitlab.com/CE4021/Group2/week4/raw/CathalCronin/Test_Results.png)


In [2]:
"""
Test Matrix class pulled from week3 repo using Mark Murnane's branch as a starting point
as required via assignment specification. Class names and references here are referencing etivity 3

I have added to these tests the extra requirements for Etivitiy 4
"""

import unittest

from tensor import Tensor


class TestMatrixImplementationEtivity3(unittest.TestCase):
    """Tests the implementation of the Matrix class for Etivity 3.

    Unit tests are:
        1) Create a new matrix and confirm size is correctly reported in a tuple
        2) Add two matrices
        3) Subtract two matrices
        5) Multiply a matrix by a scalar
        6) Multiply a matrix by a vector
        7) Multiply a matrix by a matrix
        8) Test equality and inequality

    Edge Cases in this Unit Test are:
        1) Create a new matrix with an empty list or non-list parameter
        2) Create a matrix with non-list rows
        3) Create a matrix with empty rows
        4) Create a matrix with rows that are not the same length
        5) Perform addition/subtraction of a matrix and a non-matrix
        6) Perform addition/subtraction of a matrix and an incompatible matrix
        7) Multiply a matrix by a non-numeric value
        8) Multiply a matrix by an incompatible matrix

        ========================================================================

        Added 3 more tests to show that determinant, inverse and cross product are all working.
    """

    # Internal matrix and (column) vector representations for test cases
    def setUp(self):
        self._A_2x2 = ((1, 3), (5, 7))
        self._B_2x2 = ((2, 4), (6, 8))
        self._C_2x2 = ((4, 7), (2, 6))

        self._E_vector = ([1], [2], [3])
        self._F_vector = ([1], [5], [7])

        self._Err_A_2x2 = [[1, 3], ["5", 7]]
        self._Err_B_2x2 = [[2, 4], []]
        self._Err_C_2x2 = [[1, 2], [4, 5, 6]]
        self._Err_D = [[3, 4], 5]

        self._B_2x1 = [[2], [1]]

        self._A_3x3 = [[1, 2, 3], [0, 3, 4], [5, 6, 8]]
        self._B_3x3 = [[1, 3, 6], [2, 1, 2], [4, 2, 1]]

        self._B_3x1 = [[2], [1], [3]]

        self._B_3x2 = [[4, 7], [3, 9], [5, 2]]

    def test_matrix_creation(self):
        # Checks that a matrix is created successfully with the correct dimensions and content
        a = Tensor(self._A_2x2)
        self.assertEqual(a.get_order(), (2, 2))
        self.assertEqual(a.elements, self._A_2x2)

    def test_matrix_invalid_creation(self):
        # Checks that a matrix can't be created without a valid element list
        self.assertRaises(TypeError, Tensor, None)
        self.assertRaises(TypeError, Tensor, [])
        self.assertRaises(TypeError, Tensor, self._Err_D)
        self.assertRaises(ValueError, Tensor, self._Err_A_2x2)
        self.assertRaises(ValueError, Tensor, self._Err_B_2x2)
        self.assertRaises(ValueError, Tensor, self._Err_C_2x2)

    def test_matrix_add_matrices(self):
        # Checks that two same size matrices are added together correctly
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_2x2)
        c = a + b
        expected = ((3, 7), (11, 15))
        self.assertEqual(c.elements, expected)

    def test_matrix_add_invalid_matrices(self):
        # Checks that two same differing size matrices can't be added or subtracted
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_3x3)

        func = lambda x, y: x + y

        self.assertRaises(TypeError, func, a, b)
        self.assertRaises(TypeError, func, a, 3)
        self.assertRaises(TypeError, func, a, "")

    def test_matrix_subtract_matrices(self):
        # Checks that two same size matrices are added together correctly
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_2x2)
        c = a - b
        expected = ((-1, -1), (-1, -1))
        self.assertEqual(c.elements, expected)

    def test_matrix_subtract_invalid_matrices(self):
        # Checks that two same differing size matrices can't be added or subtracted
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_3x3)

        func = lambda x, y: x - y

        self.assertRaises(TypeError, func, a, b)
        self.assertRaises(TypeError, func, a, 3)
        self.assertRaises(TypeError, func, a, "")

    def test_matrix_multiply_by_scalar(self):
        # Checks the correct multiplication of a matrix by a scalar
        a = Tensor(self._B_3x2)
        b = a * 3
        expected = ((12, 21), (9, 27), (15, 6))
        self.assertEqual(b.elements, expected)

    def test_matrix_multiply_by_vector(self):
        # Checks the correct multiplication of a matrix by a vector
        a = Tensor(self._B_2x2)
        b = Tensor(self._B_2x1)
        c = a @ b
        expected = ((8,), (20,))
        self.assertEqual(c.elements, expected)

    def test_matrix_multiply_same_order_matrices(self):
        # Checks that two same size matrices are multiplied together correctly
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_2x2)
        c = a @ b
        expected = ((20, 28), (52, 76))
        self.assertEqual(c.elements, expected)

    def test_matrix_multiply_nonsquare_matrices(self):
        # Checks that two different size matrices are multiplied together correctly
        a = Tensor(self._A_3x3)
        b = Tensor(self._B_3x2)
        c = a @ b
        expected = ((25, 31), (29, 35), (78, 105))
        self.assertEqual(c.elements, expected)

    def test_matrix_multiply_incompatible_matrices(self):
        # Checks that matrices with incompatible dimension/order can't be multiplied
        a = Tensor(self._A_3x3)
        b = Tensor(self._B_3x2)

        func = lambda x, y: x * y

        self.assertRaises(TypeError, func, b, a)

    def test_matrix_multiply_non_numeric(self):
        # Checks that matrices can't be multiplied by a string or non-Tensor object
        a = Tensor(self._A_2x2)

        func = lambda x, y: x * y

        self.assertRaises(TypeError, func, a, "")
        self.assertRaises(TypeError, func, a, None)
        self.assertRaises(TypeError, func, a, [])

    def test_matrix_equality(self):
        # Check that matrix equality rules are correct
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_2x2)
        c = Tensor([[1, 3], [5, 7]])
        d = Tensor(self._A_3x3)

        self.assertTrue(a != b)
        self.assertTrue(a == c)
        self.assertFalse(a == d)
        self.assertFalse(c == d)

    # ===============================================================
    # Tests for Etivity 4 based on persons code from etivity 3
    #
    # Requirements from SULIS:
    #   - Sum of a 4x4 with a 4x4 matrix
    #   - Multiplication of a 4x4 with a 4x4 matrix
    #   - Multiplication of a 4x4 matrix with a suitable vector
    #   - Determinant of a 2x2 matrix
    #   - Inverse of a 2x2 matrix
    #   - Cross-product of 2 suitable tensors

    def test_sum_4_by_4_matrix(self):
        a = Tensor([[2, 3, 2, 9], [8, 3, 1, 7], [4, 1, 9, 5], [2, 4, 0, 9]])
        b = Tensor([[2, 3, 5, 2], [1, 9, 1, 7], [6, 3, 9, 4], [5, 2, 4, 7]])
        expected_result = Tensor([[4, 6, 7, 11], [9, 12, 2, 14], [10, 4, 18, 9], [7, 6, 4, 16]])

        result = a + b

        self.assertEqual(result, expected_result)

    def test_multiply_4_by_4_matrix(self):
        a = Tensor([[2, 3, 2, 9], [8, 3, 1, 7], [4, 1, 9, 5], [2, 4, 0, 9]])
        b = Tensor([[2, 3, 5, 2], [1, 9, 1, 7], [6, 3, 9, 4], [5, 2, 4, 7]])
        expected_result = Tensor([[64, 57, 67, 96], [60, 68, 80, 90], [88, 58, 122, 86], [53, 60, 50, 95]])

        result = a @ b

        self.assertEqual(result, expected_result)

    def test_scalar_multiplication_4_by_4(self):
        a = Tensor([[2, 3, 2, 9], [8, 3, 1, 7], [4, 1, 9, 5], [2, 4, 0, 9]])
        v = Tensor([[2], [1], [6], [5]])
        expected_result = Tensor([[64], [60], [88], [53]])

        result = a @ v

        self.assertEqual(result, expected_result)

    def test_matrix_determinent(self):
        a = Tensor(self._A_2x2)
        det_a = a.determinant()
        expected_det = -8
        self.assertEqual(expected_det, det_a)

    def test_matrix_inverse(self):
        c = Tensor(self._C_2x2)
        inverse_c = c.inverse()
        expected_inverse = Tensor([(0.6, -0.7), (-0.2, 0.4)])

        self.assertEqual(expected_inverse, inverse_c)

    def test_matrix_cross_product(self):
        vector_e = Tensor(self._E_vector)
        vector_f = Tensor(self._F_vector)
        expected_cross_product = Tensor(([-1], [-4], [3]))
        actual_cross_product = vector_e.cross_product(vector_f)

        self.assertEqual(expected_cross_product, actual_cross_product)
